In [1]:
import numpy as np
import pandas as pd


sustainability_path = "backend/data/Dataset 1 (Sustainability Research Results).xlsx"

sustainability_df = pd.read_excel(sustainability_path, engine='openpyxl')

sustainability_df

,Unnamed: 0,Total Sample,Unnamed: 2,Gender,Unnamed: 4,Unnamed: 5,UK County,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144,Unnamed: 145,Unnamed: 146
0,Demographics,NaN,NaN,Male,Female,,North East,North West,Yorkshire & The Humber,East Midlands,...,Lush,Lidl,Ecover,Oceansaver,Tesco,No / Don't Know,Tesla,Nike,No Group,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gender,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Total,1009,NaN,477,532,,45,114,87,78,...,18,45,21,3,15,714,12,23,112,
4,NaN,100%,NaN,47%,53%,,4%,11%,9%,8%,...,2%,4%,2%,0%,1%,64%,1%,2%,10%,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,NaN,1%,NaN,0.025157,0,,0,0.017544,0,0.025641,...,0,0,0,0,0,0,0,0,0,
625,Nike,23,NaN,13,10,,0,4,1,0,...,0,0,0,0,0,0,0,0,0,
626,NaN,2%,NaN,0.027254,0.018797,,0,0.035088,0.011494,0,...,0,0,0,0,0,0,0,0,0,
627,No Group,112,NaN,57,55,,4,14,12,5,...,0,0,0,0,0,0,0,0,0,


In [2]:
def extract_data_blocks(df):
    # Identify the rows where questions start
    question_rows = df[df.iloc[:, 0].str.contains('Question', na=False)].index

    # Create a dictionary to store the split dataframes
    dfs = {}

    # Iterate over the question rows and split the dataframe
    for i, start_row in enumerate(question_rows):
        end_row = question_rows[i + 1] if i + 1 < len(question_rows) else len(df)
        question_text = df.iloc[start_row, 0]
        dfs[question_text] = df.iloc[start_row:end_row].reset_index(drop=True)#.fillna('')
    
    return dfs

In [3]:
sustainability_dfs = extract_data_blocks(sustainability_df)

len(sustainability_dfs)

21

In [4]:
sustainability_dfs

{'Questions':   Unnamed: 0 Total Sample  Unnamed: 2 Gender Unnamed: 4 Unnamed: 5 UK County  \
 0  Questions          NaN         NaN    NaN        NaN        NaN       NaN   
 1        NaN          NaN         NaN    NaN        NaN        NaN       NaN   
 
   Unnamed: 7 Unnamed: 8 Unnamed: 9  ... Unnamed: 137 Unnamed: 138  \
 0        NaN        NaN        NaN  ...          NaN          NaN   
 1        NaN        NaN        NaN  ...          NaN          NaN   
 
   Unnamed: 139 Unnamed: 140 Unnamed: 141 Unnamed: 142 Unnamed: 143  \
 0          NaN          NaN          NaN          NaN          NaN   
 1          NaN          NaN          NaN          NaN          NaN   
 
   Unnamed: 144 Unnamed: 145 Unnamed: 146  
 0          NaN          NaN          NaN  
 1          NaN          NaN          NaN  
 
 [2 rows x 147 columns],
 'Question 1 - Please rank the importance of the following issues facing the UK today. From 1 being the most important to 10 being of lesser importance. Ple

In [5]:
# save the dataframes to excel files
for i, (key, df) in enumerate(sustainability_dfs.items()):
    key = "_".join(key.split(" ")[:2]).lower()
    df.to_excel(f"backend/data/processed/sustainability_{key}.xlsx", index=False)

In [6]:
christmas_path = "backend/data/Dataset 2 (Christmas Research Results).xlsx"
christmas_df = pd.read_excel(christmas_path, engine='openpyxl')

christmas_dfs = extract_data_blocks(christmas_df)

In [7]:
for i, (key, df) in enumerate(christmas_dfs.items()):
    key = "_".join(key.split(" ")[:2]).lower()
    df.to_excel(f"backend/data/processed/christmas_{key}.xlsx", index=False)

In [8]:
# test the SurveyAnalysisRAGSystem
import pandas as pd
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from backend.utils.logging import get_logger
from time import time
from glob import glob

# Initialize logger
logger = get_logger(__name__)

# Set your OpenAI API Key
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

class DocumentLoader:
    def __init__(self, file_paths):
        self.file_paths = file_paths

    def load_documents(self):
        """Load and split the documents from the Excel files using the UnstructuredExcelLoader."""
        logger.info(f"Loading documents from {len(self.file_paths)} files: {self.file_paths}")
        docs = [self._process(file_path) for file_path in self.file_paths]
        documents = sum(docs, [])  # Flatten the list of lists
        logger.info(f"Total documents loaded: {len(documents)}")
        return documents

    def _process(self, file_path):
        logger.info(f"Processing file: {file_path}")
        docs = UnstructuredExcelLoader(file_path, mode='elements').load_and_split()
        for doc in docs:
            dataset_id = file_path.split("/")[-1].split("_")[0]
            doc.metadata['dataset_id'] = dataset_id
        logger.info(f"Loaded {len(docs)} documents from {file_path}")
        return docs

class VectorStore:
    def __init__(self, documents, embedding_model):
        self.documents = documents
        self.embedding_model = embedding_model

    def create_vectorstore(self):
        """Generate embeddings for the documents and create a FAISS vectorstore for retrieval."""
        logger.info("Generating embeddings for documents")
        embeddings = OpenAIEmbeddings(model=self.embedding_model)
        vectorstore = FAISS.from_documents(self.documents, embeddings)
        logger.info(f"Vectorstore created successfully with {len(self.documents)} documents")
        return vectorstore

class PromptTemplateFactory:
    @staticmethod
    def create_prompt_template():
        """Define the prompt template for querying the model with relevant context."""
        logger.info("Defining prompt template")
        return ChatPromptTemplate(
            input_variables=['context', 'question'],
            messages=[
                HumanMessagePromptTemplate(
                    prompt=PromptTemplate(
                        input_variables=['context', 'question'],
                        template=(
                            "You are an assistant for question-answering tasks. "
                            "Specialized in data analysis and interpreting survey data. "
                            "Use the following pieces of retrieved context to answer the question. "
                            "If you don't know the answer, just say that you don't know. "
                            "Use three sentences maximum and keep the answer concise.\n"
                            "Question: {question} \n"
                            "Context: {context} \n"
                            "Answer:"
                        )
                    )
                )
            ]
        )

class SurveyAnalysisRAGSystem:
    def __init__(self, file_path, embedding_model="text-embedding-3-large", llm_model="gpt-4o-mini"):
        logger.info("Initializing SurveyAnalysisRAGSystem")
        self.file_paths = glob(file_path + "*.xlsx")
        self.embedding_model = embedding_model
        self.llm_model = llm_model
        
        # Load documents
        self.documents = DocumentLoader(self.file_paths).load_documents()

        # Create vector embeddings and store them in FAISS
        self.vectorstore = VectorStore(self.documents, self.embedding_model).create_vectorstore()

        # Initialize the language model for generating insights
        self.llm = ChatOpenAI(model=self.llm_model, temperature=0.25, max_tokens=1024)

        # Setup prompt template for query generation
        self.prompt = PromptTemplateFactory.create_prompt_template()

    @staticmethod
    def format_docs(docs):    
        return "\n\n".join(str(doc.metadata["filename"]) + "\n\n" + doc.page_content for doc in docs) 

    def generate_answer(self, query, dataset_id, k=16):
        """Generate an answer for the given query using the specified dataset."""
        logger.info(f"Generating answer for query: {query} with dataset_id: {dataset_id}")
        st = time()
        self.retriever = self.vectorstore.as_retriever(
            search_type="mmr", # mmr
            search_kwargs={'k': k, 'lambda_mult': 0.25, 'filter': {'dataset_id': dataset_id}}
        )
        self.qa_chain = (
            {
                "context": self.retriever | self.format_docs,
                "question": RunnablePassthrough(),
            }
            | self.prompt
            | self.llm
            | StrOutputParser()
        )
        
        logger.info("Invoking QA chain...")
        output = {"result" : self.qa_chain.invoke(query)}
        output["time_taken"] = time() - st
        return output


In [9]:
# Define your file paths
file_path = 'backend/data/processed/'

# Initialize the system with the file paths
logger.info("Initializing SurveyAnalysisRAGSystem with example file paths")
survey_system = SurveyAnalysisRAGSystem(file_path)

2024-10-17 20:17:19,752 - __main__ - INFO - Initializing SurveyAnalysisRAGSystem with example file paths
2024-10-17 20:17:19,753 - __main__ - INFO - Initializing SurveyAnalysisRAGSystem
2024-10-17 20:17:19,755 - __main__ - INFO - Loading documents from 29 files: ['backend/data/processed/sustainability_question_15.xlsx', 'backend/data/processed/sustainability_question_7.xlsx', 'backend/data/processed/sustainability_question_13.xlsx', 'backend/data/processed/christmas_question_1.xlsx', 'backend/data/processed/sustainability_question_9.xlsx', 'backend/data/processed/christmas_question_3.xlsx', 'backend/data/processed/sustainability_question_2.xlsx', 'backend/data/processed/sustainability_question_14.xlsx', 'backend/data/processed/sustainability_question_19.xlsx', 'backend/data/processed/sustainability_question_1.xlsx', 'backend/data/processed/sustainability_question_8.xlsx', 'backend/data/processed/sustainability_question_17.xlsx', 'backend/data/processed/christmas_question_4.xlsx', 'back

In [10]:
# Query the system with a specific question
query = "summarize results for question 1"
logger.info(f"Querying system with: {query}")

response = survey_system.generate_answer(query, dataset_id='sustainability', k=16)

# Print the response
logger.info(f"Response: {response}")

print(response["result"])

2024-10-17 20:17:40,312 - __main__ - INFO - Querying system with: summarize results for question 1
2024-10-17 20:17:40,314 - __main__ - INFO - Generating answer for query: summarize results for question 1 with dataset_id: sustainability
2024-10-17 20:17:40,316 - __main__ - INFO - Invoking QA chain...
2024-10-17 20:17:43,834 - __main__ - INFO - Response: {'result': 'The results for Question 1 indicate that respondents ranked various issues facing the UK, with a total sample of 1009 participants. The specific rankings of the issues were not provided in the context, so I cannot summarize the exact results. If you have access to the detailed rankings or data, I could help interpret that information further.', 'time_taken': 3.5185585021972656}


The results for Question 1 indicate that respondents ranked various issues facing the UK, with a total sample of 1009 participants. The specific rankings of the issues were not provided in the context, so I cannot summarize the exact results. If you have access to the detailed rankings or data, I could help interpret that information further.
